In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import RandomOverSampler
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.ensemble import RandomForestClassifier

import sqlite3


In [2]:
# Create connection

con = sqlite3.connect("stellar_class_db")
cur = con.cursor()

In [3]:
# Load the data from sqlite from stars table
stars_df = pd.read_sql_query("SELECT * FROM stars", con)
stars_df

,spec_obj_ID,alpha,delta,u,g,r,i,z,class,redshift
0,2.751763e+18,39.149691,28.102842,21.74669,20.03493,19.17553,18.81823,18.65422,STAR,-0.000008
1,7.323011e+18,345.801874,32.672868,23.17274,20.14496,19.41948,19.22034,18.89359,STAR,0.000072
2,4.822278e+18,353.201522,3.080796,24.54890,21.44267,20.95315,20.79360,20.48442,STAR,-0.000429
3,9.855073e+18,14.383135,3.214326,21.82154,20.55730,19.94918,19.76057,19.55514,STAR,-0.000440
4,8.237292e+18,144.785293,46.826496,24.54793,22.33601,20.92259,19.87177,19.16934,STAR,-0.000120
...,...,...,...,...,...,...,...,...,...,...
21589,8.616681e+18,338.557124,24.678940,26.03642,23.55800,21.91035,20.74742,19.72344,STAR,-0.000070
21590,7.336558e+18,355.651889,27.734322,24.87293,20.71855,19.83588,19.54439,19.43266,STAR,-0.000778
21591,1.229506e+19,251.031744,31.840055,23.25934,22.93305,21.72368,20.44625,19.73752,STAR,-0.000153
21592,3.238115e+18,172.281648,-8.909060,22.26657,20.76710,20.22759,20.08172,19.98396,STAR,-0.000395


In [4]:
stars_df.set_index(['spec_obj_ID'], inplace = True)

In [5]:
stars_df

,alpha,delta,u,g,r,i,z,class,redshift
spec_obj_ID,,,,,,,,,
2.751763e+18,39.149691,28.102842,21.74669,20.03493,19.17553,18.81823,18.65422,STAR,-0.000008
7.323011e+18,345.801874,32.672868,23.17274,20.14496,19.41948,19.22034,18.89359,STAR,0.000072
4.822278e+18,353.201522,3.080796,24.54890,21.44267,20.95315,20.79360,20.48442,STAR,-0.000429
9.855073e+18,14.383135,3.214326,21.82154,20.55730,19.94918,19.76057,19.55514,STAR,-0.000440
8.237292e+18,144.785293,46.826496,24.54793,22.33601,20.92259,19.87177,19.16934,STAR,-0.000120
...,...,...,...,...,...,...,...,...,...
8.616681e+18,338.557124,24.678940,26.03642,23.55800,21.91035,20.74742,19.72344,STAR,-0.000070
7.336558e+18,355.651889,27.734322,24.87293,20.71855,19.83588,19.54439,19.43266,STAR,-0.000778
1.229506e+19,251.031744,31.840055,23.25934,22.93305,21.72368,20.44625,19.73752,STAR,-0.000153


In [29]:
 #stars_df = stars_df.drop(['run_ID','rerun_ID', 'cam_col', 'field_ID', "obj_ID",'plate', 'mjd', 'fiber_ID'], axis = 1 )

In [30]:
#stars_df

,alpha,delta,u,g,r,i,z,class,redshift
spec_obj_ID,,,,,,,,,
2.751763e+18,39.149691,28.102842,21.74669,20.03493,19.17553,18.81823,18.65422,STAR,-0.000008
7.323011e+18,345.801874,32.672868,23.17274,20.14496,19.41948,19.22034,18.89359,STAR,0.000072
4.822278e+18,353.201522,3.080796,24.54890,21.44267,20.95315,20.79360,20.48442,STAR,-0.000429
9.855073e+18,14.383135,3.214326,21.82154,20.55730,19.94918,19.76057,19.55514,STAR,-0.000440
8.237292e+18,144.785293,46.826496,24.54793,22.33601,20.92259,19.87177,19.16934,STAR,-0.000120
...,...,...,...,...,...,...,...,...,...
8.616681e+18,338.557124,24.678940,26.03642,23.55800,21.91035,20.74742,19.72344,STAR,-0.000070
7.336558e+18,355.651889,27.734322,24.87293,20.71855,19.83588,19.54439,19.43266,STAR,-0.000778
1.229506e+19,251.031744,31.840055,23.25934,22.93305,21.72368,20.44625,19.73752,STAR,-0.000153


In [31]:
stars_df.to_sql("stars", con, if_exists="replace")

In [51]:
df.to_csv("machine_learning_model.csv")

In [14]:
df["z"].min()

NameError: name 'df' is not defined

In [15]:
frequency = [1, 2, 3]

u_size_bins = [-10000, -6600,-3310, 33]
stars_df["binned_u"] = pd.cut(stars_df["u"], u_size_bins, labels = frequency)

g_size_bins = [-10000, -6600,-3310, 33]
stars_df["binned_g"] = pd.cut(stars_df["g"], g_size_bins, labels = frequency)

r_size_bins = [9,16,23, 30]
stars_df["binned_r"] = pd.cut(stars_df["r"], r_size_bins, labels = frequency)

i_size_bins = [9,17,25, 33]
stars_df["binned_i"] = pd.cut(stars_df["i"], i_size_bins, labels = frequency)

z_size_bins = [-10000, -6600,-3310, 30]
stars_df["binned_z"] = pd.cut(stars_df["z"], z_size_bins, labels = frequency)

stars_df.sample(5)

,alpha,delta,u,g,r,i,z,class,redshift,binned_u,binned_g,binned_r,binned_i,binned_z
spec_obj_ID,,,,,,,,,,,,,,
6.531566e+18,147.410784,30.356104,24.56459,21.95643,21.62675,19.97989,21.30295,STAR,0.000036,3,3,2,2,3
3.667162e+18,151.449705,3.086991,20.68095,19.59083,19.21626,19.08393,19.02754,STAR,0.000079,3,3,2,2,3
3.704780e+17,176.641543,-3.648650,17.47629,16.27715,15.84659,15.69811,15.65250,STAR,0.000188,3,3,1,1,3
4.360716e+18,217.483874,29.451591,23.33428,22.23430,20.74029,19.51495,18.86996,STAR,-0.000093,3,3,2,2,3
9.068123e+18,250.635803,47.968386,23.53882,22.14318,21.71372,22.82896,20.12123,STAR,-0.000422,3,3,2,2,3


In [17]:
# Load the data from sqlite from quasars table
quasars_df = pd.read_sql_query("SELECT * FROM quasars", con)
quasars_df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,mjd,fiber_ID
0,1.237680e+18,340.995121,20.589476,23.48827,23.33776,21.32195,20.25615,19.54544,8102,301,3,110,5.658977e+18,QSO,1.424659,5026,55855,741
1,1.237679e+18,23.234926,11.418188,21.46973,21.17624,20.92829,20.60826,20.42573,7773,301,2,462,1.246262e+19,QSO,0.586455,11069,58456,113
2,1.237679e+18,1.494389,3.291746,20.38562,20.40514,20.29996,20.05918,19.89044,7712,301,5,339,9.843824e+18,QSO,2.031528,8743,57663,295
3,1.237658e+18,145.883006,47.300484,21.73992,21.53095,21.26763,21.36257,21.15861,2821,301,2,33,8.228240e+18,QSO,2.075680,7308,56709,596
4,1.237679e+18,20.052556,11.497881,21.89214,21.35124,21.18755,20.84300,20.76580,7773,301,2,441,1.246830e+19,QSO,1.528308,11074,58488,306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18956,1.237655e+18,134.347759,47.771911,24.17897,22.89025,21.26451,20.94941,19.87500,2243,301,4,128,8.458979e+18,QSO,0.219966,7513,56780,339
18957,1.237662e+18,222.761686,32.203212,20.23421,19.76480,19.46940,19.36135,19.21768,3900,301,1,583,4.356138e+18,QSO,0.398574,3869,55273,112
18958,1.237656e+18,259.504325,31.462416,24.86685,23.22772,21.82982,21.80359,21.56733,2335,301,5,107,1.312146e+19,QSO,1.239638,11654,58543,812
18959,1.237661e+18,217.958430,52.316738,24.15617,22.05986,21.80826,21.95129,21.24179,3705,301,2,124,7.916218e+18,QSO,1.067543,7031,56449,58


In [18]:
quasars_df.set_index(['spec_obj_ID'], inplace = True)

In [19]:
quasars_df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,class,redshift,plate,mjd,fiber_ID
spec_obj_ID,,,,,,,,,,,,,,,,,
5.658977e+18,1.237680e+18,340.995121,20.589476,23.48827,23.33776,21.32195,20.25615,19.54544,8102,301,3,110,QSO,1.424659,5026,55855,741
1.246262e+19,1.237679e+18,23.234926,11.418188,21.46973,21.17624,20.92829,20.60826,20.42573,7773,301,2,462,QSO,0.586455,11069,58456,113
9.843824e+18,1.237679e+18,1.494389,3.291746,20.38562,20.40514,20.29996,20.05918,19.89044,7712,301,5,339,QSO,2.031528,8743,57663,295
8.228240e+18,1.237658e+18,145.883006,47.300484,21.73992,21.53095,21.26763,21.36257,21.15861,2821,301,2,33,QSO,2.075680,7308,56709,596
1.246830e+19,1.237679e+18,20.052556,11.497881,21.89214,21.35124,21.18755,20.84300,20.76580,7773,301,2,441,QSO,1.528308,11074,58488,306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8.458979e+18,1.237655e+18,134.347759,47.771911,24.17897,22.89025,21.26451,20.94941,19.87500,2243,301,4,128,QSO,0.219966,7513,56780,339
4.356138e+18,1.237662e+18,222.761686,32.203212,20.23421,19.76480,19.46940,19.36135,19.21768,3900,301,1,583,QSO,0.398574,3869,55273,112
1.312146e+19,1.237656e+18,259.504325,31.462416,24.86685,23.22772,21.82982,21.80359,21.56733,2335,301,5,107,QSO,1.239638,11654,58543,812


In [20]:
quasars_df = quasars_df.drop(['run_ID','rerun_ID', 'cam_col', 'field_ID', "obj_ID",'plate', 'mjd', 'fiber_ID'], axis = 1 )

In [21]:
quasars_df

,alpha,delta,u,g,r,i,z,class,redshift
spec_obj_ID,,,,,,,,,
5.658977e+18,340.995121,20.589476,23.48827,23.33776,21.32195,20.25615,19.54544,QSO,1.424659
1.246262e+19,23.234926,11.418188,21.46973,21.17624,20.92829,20.60826,20.42573,QSO,0.586455
9.843824e+18,1.494389,3.291746,20.38562,20.40514,20.29996,20.05918,19.89044,QSO,2.031528
8.228240e+18,145.883006,47.300484,21.73992,21.53095,21.26763,21.36257,21.15861,QSO,2.075680
1.246830e+19,20.052556,11.497881,21.89214,21.35124,21.18755,20.84300,20.76580,QSO,1.528308
...,...,...,...,...,...,...,...,...,...
8.458979e+18,134.347759,47.771911,24.17897,22.89025,21.26451,20.94941,19.87500,QSO,0.219966
4.356138e+18,222.761686,32.203212,20.23421,19.76480,19.46940,19.36135,19.21768,QSO,0.398574
1.312146e+19,259.504325,31.462416,24.86685,23.22772,21.82982,21.80359,21.56733,QSO,1.239638


In [22]:
quasars_df.to_sql("quasars", con, if_exists="replace")

In [32]:
frequency = [1, 2, 3]

u_size_bins = [-10000, -6600,-3310, 33]
quasars_df["binned_u"] = pd.cut(quasars_df["u"], u_size_bins, labels = frequency)

g_size_bins = [-10000, -6600,-3310, 33]
quasars_df["binned_g"] = pd.cut(quasars_df["g"], g_size_bins, labels = frequency)

r_size_bins = [9,16,23, 30]
quasars_df["binned_r"] = pd.cut(quasars_df["r"], r_size_bins, labels = frequency)

i_size_bins = [9,17,25, 33]
quasars_df["binned_i"] = pd.cut(quasars_df["i"], i_size_bins, labels = frequency)

z_size_bins = [-10000, -6600,-3310, 30]
quasars_df["binned_z"] = pd.cut(quasars_df["z"], z_size_bins, labels = frequency)

quasars_df.sample(5)

,alpha,delta,u,g,r,i,z,class,redshift,binned_u,binned_g,binned_r,binned_i,binned_z
spec_obj_ID,,,,,,,,,,,,,,
7.572868e+18,224.263448,50.031854,20.66810,20.19139,19.82914,19.80626,19.52610,QSO,2.325700,3,3,2,2,3
6.953714e+18,0.847899,13.564264,26.08850,21.26539,20.33314,20.03217,19.96611,QSO,3.563044,3,3,2,2,3
4.475723e+18,195.684663,37.332758,22.91049,21.99886,20.69863,19.91150,19.49593,QSO,0.556629,3,3,2,2,3
8.841798e+18,2.535373,1.640446,19.79256,19.56019,19.28005,19.24887,19.10384,QSO,1.056582,3,3,2,2,3
5.840285e+18,253.690029,40.070439,21.87903,20.52036,19.91514,19.56784,19.04740,QSO,2.441414,3,3,2,2,3


In [33]:
quasars_df

,alpha,delta,u,g,r,i,z,class,redshift,binned_u,binned_g,binned_r,binned_i,binned_z
spec_obj_ID,,,,,,,,,,,,,,
5.658977e+18,340.995121,20.589476,23.48827,23.33776,21.32195,20.25615,19.54544,QSO,1.424659,3,3,2,2,3
1.246262e+19,23.234926,11.418188,21.46973,21.17624,20.92829,20.60826,20.42573,QSO,0.586455,3,3,2,2,3
9.843824e+18,1.494389,3.291746,20.38562,20.40514,20.29996,20.05918,19.89044,QSO,2.031528,3,3,2,2,3
8.228240e+18,145.883006,47.300484,21.73992,21.53095,21.26763,21.36257,21.15861,QSO,2.075680,3,3,2,2,3
1.246830e+19,20.052556,11.497881,21.89214,21.35124,21.18755,20.84300,20.76580,QSO,1.528308,3,3,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8.458979e+18,134.347759,47.771911,24.17897,22.89025,21.26451,20.94941,19.87500,QSO,0.219966,3,3,2,2,3
4.356138e+18,222.761686,32.203212,20.23421,19.76480,19.46940,19.36135,19.21768,QSO,0.398574,3,3,2,2,3
1.312146e+19,259.504325,31.462416,24.86685,23.22772,21.82982,21.80359,21.56733,QSO,1.239638,3,3,2,2,3


In [34]:
# Load the data from sqlite from galaxies table
galaxies_df = pd.read_sql_query("SELECT * FROM galaxies", con)
galaxies_df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,mjd,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59440,1.237679e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,1.055431e+19,GALAXY,0.000000,9374,57749,438
59441,1.237679e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,8.586351e+18,GALAXY,0.404895,7626,56934,866
59442,1.237668e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,3.112008e+18,GALAXY,0.143366,2764,54535,74
59443,1.237661e+18,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,3650,301,4,131,7.601080e+18,GALAXY,0.455040,6751,56368,470


In [35]:
galaxies_df.set_index(['spec_obj_ID'], inplace = True)

In [36]:
galaxies_df = galaxies_df.drop(['run_ID','rerun_ID', 'cam_col', 'field_ID', "obj_ID",'plate', 'mjd', 'fiber_ID'], axis = 1 )

In [37]:
galaxies_df.to_sql("galaxies", con, if_exists="replace")

In [38]:
frequency = [1, 2, 3]

u_size_bins = [-10000, -6600,-3310, 33]
galaxies_df["binned_u"] = pd.cut(galaxies_df["u"], u_size_bins, labels = frequency)

g_size_bins = [-10000, -6600,-3310, 33]
galaxies_df["binned_g"] = pd.cut(galaxies_df["g"], g_size_bins, labels = frequency)

r_size_bins = [9,16,23, 30]
galaxies_df["binned_r"] = pd.cut(galaxies_df["r"], r_size_bins, labels = frequency)

i_size_bins = [9,17,25, 33]
galaxies_df["binned_i"] = pd.cut(galaxies_df["i"], i_size_bins, labels = frequency)

z_size_bins = [-10000, -6600,-3310, 30]
galaxies_df["binned_z"] = pd.cut(galaxies_df["z"], z_size_bins, labels = frequency)

galaxies_df.sample(5)

,alpha,delta,u,g,r,i,z,class,redshift,binned_u,binned_g,binned_r,binned_i,binned_z
spec_obj_ID,,,,,,,,,,,,,,
7.661816e+18,206.579261,56.157426,26.06968,22.23723,20.72961,19.71656,19.23207,GALAXY,0.549101,3,3,2,2,3
1.008254e+19,115.227141,20.371441,22.84854,22.66115,22.68279,21.85423,22.12852,GALAXY,0.974616,3,3,2,2,3
4.862906e+18,333.707643,4.391110,23.28140,20.55934,18.95243,18.27040,17.89017,GALAXY,0.245924,3,3,2,2,3
4.700657e+18,255.782380,19.418125,24.97690,21.61855,19.74158,18.83133,18.39075,GALAXY,0.462539,3,3,2,2,3
5.313253e+18,229.439612,32.013928,24.31543,22.17832,21.83941,21.32130,22.01619,GALAXY,0.384823,3,3,2,2,3


In [39]:
galaxies_df

,alpha,delta,u,g,r,i,z,class,redshift,binned_u,binned_g,binned_r,binned_i,binned_z
spec_obj_ID,,,,,,,,,,,,,,
6.543777e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,GALAXY,0.634794,3,3,2,2,3
1.176014e+19,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,GALAXY,0.779136,3,3,2,2,3
5.152200e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,GALAXY,0.644195,3,3,2,2,3
1.030107e+19,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,GALAXY,0.932346,3,3,2,2,3
6.891865e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,GALAXY,0.116123,3,3,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.055431e+19,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,GALAXY,0.000000,3,3,2,2,3
8.586351e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,GALAXY,0.404895,3,3,2,2,3
3.112008e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,GALAXY,0.143366,3,3,2,2,3


In [50]:
# Join stars, quasars, and galaxies tables 
df = pd.read_sql_query("SELECT * FROM stars INNER JOIN quasars ON stars.spec_obj_ID=quasars.spec_obj_ID", con)
    



In [51]:
df

,spec_obj_ID,alpha,delta,u,g,r,i,z,class,redshift,spec_obj_ID,alpha,delta,u,g,r,i,z,class,redshift


In [52]:
stars_df

,alpha,delta,u,g,r,i,z,class,redshift
spec_obj_ID,,,,,,,,,
2.751763e+18,39.149691,28.102842,21.74669,20.03493,19.17553,18.81823,18.65422,STAR,-0.000008
7.323011e+18,345.801874,32.672868,23.17274,20.14496,19.41948,19.22034,18.89359,STAR,0.000072
4.822278e+18,353.201522,3.080796,24.54890,21.44267,20.95315,20.79360,20.48442,STAR,-0.000429
9.855073e+18,14.383135,3.214326,21.82154,20.55730,19.94918,19.76057,19.55514,STAR,-0.000440
8.237292e+18,144.785293,46.826496,24.54793,22.33601,20.92259,19.87177,19.16934,STAR,-0.000120
...,...,...,...,...,...,...,...,...,...
8.616681e+18,338.557124,24.678940,26.03642,23.55800,21.91035,20.74742,19.72344,STAR,-0.000070
7.336558e+18,355.651889,27.734322,24.87293,20.71855,19.83588,19.54439,19.43266,STAR,-0.000778
1.229506e+19,251.031744,31.840055,23.25934,22.93305,21.72368,20.44625,19.73752,STAR,-0.000153


In [54]:
# Create our features
X = df.drop('class', axis=1)

# Create our target
y = df['class']

In [55]:
df.to_csv("mlm_results.csv")

In [14]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [15]:
# Resample the training data with the RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'STAR': 44608, 'GALAXY': 44608, 'QSO': 44608})

In [16]:
# Train the Logistic Regression Model using the resampled data
model = RandomForestClassifier(random_state=1)
model.fit(X_resampled, y_resampled)

RandomForestClassifier(random_state=1)

In [17]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9745176569907222

In [18]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

     GALAXY       0.98      0.98      0.97      0.98      0.98      0.95     14837
        QSO       0.95      0.94      0.99      0.95      0.97      0.93      4796
       STAR       0.98      1.00      1.00      0.99      1.00      1.00      5367

avg / total       0.98      0.98      0.98      0.98      0.98      0.96     25000



In [19]:
# Display the confusion matrix
c= confusion_matrix(y_test,y_pred)
cm = pd.DataFrame(c, index=["Actual 0", "Actual 1", "Actual 2"], columns=["Predicted 0", "Predicted 1", "Predicted 2"])
cm

,Predicted 0,Predicted 1,Predicted 2
Actual 0,14525,216,96
Actual 1,264,4532,0
Actual 2,2,0,5365


In [56]:
# Write new df to new SQLite table - 

df.to_sql("mlm_results", con, if_exists="replace")

In [57]:
# Close connection
con.close()

In [58]:
import pandas as pd
from config import password
from sqlalchemy import create_engine
engine = create_engine("postgresql://postgres:" + password + "@localhost:5432/Stellar_Classification")

In [59]:
df = pd.read_csv("mlm_results.csv")
df.to_sql("mlm_results", engine, if_exists="replace")